(ch:working_with_keras)=
# 케라스 모델 고급 활용법

**감사의 글**

아래 내용은 프랑소와 숄레의 
[Deep Learning with Python(2판)](https://github.com/fchollet/deep-learning-with-python-notebooks)의 
소스코드 내용을 참고해서 작성되었습니다.
자료를 공개한 저자에게 진심어린 감사를 전합니다.

**소스코드**

여기서 언급되는 코드를
[(구글 코랩) 케라스 모델 활용법](https://colab.research.google.com/github/codingalzi/dlp2/blob/master/notebooks/NB-working_with_keras.ipynb)에서 
직접 실행할 수 있다.

**슬라이드**

본문 내용을 요약한 [슬라이드](https://github.com/codingalzi/dlp2/raw/master/slides/slides-working_with_keras.pdf)를 다운로드할 수 있다.

**주요 내용**

- 다양한 모델 구성법
- 모델과 층의 재활용
- 모델 훈련 옵션: 콜백과 텍서보드

## 케라스 모델 구성법 1: `Sequential` 모델 활용

케라스를 이용하여 매우 간단한 방식부터 매우 복잡한 방식까지 다양한 방식으로 
필요한 수준의 모델을 구성할 수 있다.
또한 케라스의 모델과 층은 모두 각각 `Model` 클래스와 `Layer` 클래스를 상속하기에 
다른 모델 또는 층과의 융합도 용이하다.

케라스를 이용하여 세 가지 방식으로 딥러닝 모델을 구성할 수 있다.

- `Sequential` 모델 활용: 층으로 스택을 쌓아 만든 모델
- 함수형 API 활용: 가장 많이 사용됨.
- 모델 서브클래싱: 모든 것을 사용자가 지정.

먼저 `Sequential` 모델은 층으로 스택을 쌓아 만든 모델이며 가장 단순하다.

- 한 종류의 입력값과 한 종류의 출력값만 사용 가능
    - 순전파: 지정된 층의 순서대로 적용

**`Sequential` 클래스**

```python
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(64, activation="relu"),
    layers.Dense(10, activation="softmax")
])
```

층을 스택에 추가하기 위해 `add` 메서드를 이용할 수도 있다.
더해진 순서대로 층이 쌓인다.
예를 들어, 아래 코드는 앞서 정의한 모델과 동일한 모델을 구성한다.

```python
model = keras.Sequential()
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(10, activation="softmax"))
```

**모델의 가중치와 `build()` 메서드**

지금 당장 모델에 포함된 각각의 층에서 데이터 변환(순전파)를 위해 사용되는 가중치와 편향을 확인하려 하면 오류가 발생한다.

```python
>>> model.weights
...
ValueError: Weights for model sequential_1 have not yet been created. 
Weights are created when the Model is first called on inputs or 
`build()` is called with an `input_shape`.
```

이유는 가중치 행렬의 크기와 편향 벡터의 길이가 정해지지 않았기 때문이다.
지금까지는 훈련을 시작하면서 가장 먼저 층별로 가중치 행렬은 임의로, 편향 텐서는 0으로
초기화된다고만 설명했다.
이것이 가능했던 이유는 
훈련이 시작되어 입력 데이터셋이 전달될 때 입력 샘플에 포함된 특성의 개수가 첫째 층에 알려지고,
그 정보와 함께 층별 유닛수를 이용하여 가중치 행렬과 편향 벡터의 모양을 정할 수 있기 때문이다.

층별 가중치 행렬과 편향 벡터의 초기화는 앞서 설명한 대로
각 층의 `build()` 메서드가 담당한다. 
아래 코드는 {numref}`%s장 <ch:keras-tf>`에서 선언한
`SimpleDense`에서의 `build()` 메서드를 보여준다.
이미 설명하대로 첫째 층에 입력값이 처음 들어올 때 알려지는 입력값의 특성 수와
층에 포함된 유닛 수를 이용하여 가중치 행렬과 편향 벡터를 초기화 한다.

```python
def build(self, input_shape):
    input_dim = input_shape[-1]   # 입력 샘플의 특성 수
    self.W = self.add_weight(shape=(input_dim, self.units),
                             initializer="random_normal")
    self.b = self.add_weight(shape=(self.units,),
                             initializer="zeros")
```

모델 역시 `build()` 메서드를 가지며, 해당 메서드를 적절한 인자와 함께 호출하면
모델에 포함된 모든 층에 포함된 `build()` 메서드가 적절한 인자와 함게 호출되어
층별로 가중치 행렬과 편향 벡터를 초기화된다.

- `input_shape` 키워드 인자: 배치 입력 데이터셋 텐서의 모양인 `(None, 특성수)` 와 같은 인자 사용
- `None`은 배치의 크기는 중요하지 않음을 의미함, 즉 임의의 크기의 배치 데이터셋 처리 가능.

```python
>>> model.build(input_shape=(None, 3))
````

위 모델에 대해서 층별로 가중치 행렬과 편향 벡터 하나씩 총 4 개의 텐서가 생성된다.
생성된 모든 텐서를 항목으로 갖는 리스트를 모델의 `weights` 속성이 가리킨다.

```python
>>> len(model.weights)
4
```

- 1층의 가중치 행렬과 편향 벡터

```python
>>> model.weights[0].shape # 가중치 행렬
TensorShape([3, 64])

>>> model.weights[1].shape # 편향 벡터
TensorShape([64])
```

- 2층의 가중치 행렬과 편향 벡터

```python
>>> model.weights[2].shape # 가중치 행렬
TensorShape([64, 10])

>>> model.weights[3].shape # 편향 벡터
TensorShape([10])
```

**`summary()` 메서드**

완성된 모델의 층의 구조와 각 층의 출력값의 모양 정보를 확인할 수 있다.

- 모델과 층의 이름: 특별히 지정하지 않으면 기본 이름 자동으로 할당됨.
- 층별 출력값의 모양(`Output Shape`): 유닛수 확인. `None` 은 임의의 배치크기를 가리킴.
- 층별 파라미터 수(`Param #`): 훈련을 통해 학습되어야 하는 파라미터들의 개수
    - 가중치 행렬과 편향 벡터에 포함된 항목들의 수를 가리킴.
- 파라미터 수
    - `Total params`: 파라미터의 총 개수
    - `Trainable params`: 가중치 행렬과 편향 벡터에 포함된 파라리터처럼 훈련을 통해 학습(업데이트)되는 파라미터 총 개수
    - `Non-trainable params`: 훈련에는 관여하지만 훈련을 통해 학습되지는 않는 파라미터. 
        지금까지 다룬 모델에서는 비학습 파라미터 사용되지 않았음.
        나중에 비학습 파라미터 예제를 다룰 것임 ({numref}`%s장 <ch:computer-vision-advanced>` 참고).

```python
>>> model.summary()
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
dense_2 (Dense)              (None, 64)                256       
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
=================================================================
Total params: 906
Trainable params: 906
Non-trainable params: 0
_________________________________________________________________
```

**`Input()` 함수**

모델 훈련을 시작하거나 모델의 `build()` 메서드를 실행하기 전까지는 모델의 `summary()` 메서드가 작동하지 않는다.
그리고 그 이유는 입력값의 모양, 즉 특성 수를 미리 알지 못하기 때문이었다.

그런데 모델을 구성할 때 `Input()` 함수를 활용하면 모델 훈련에 사용될 
입력값에 대한 정보를 모델에 미리 전달할 수 있다.
예를 들어 아래 코드는 훈련셋의 샘플이 3 개의 특성을 갖는다는 것을 가리킨다.
물론, 이렇게 입력값의 정보를 미리 지정하면 지정된 모양의 훈련셋만 사용할 수 있다.

```python
model = keras.Sequential()
model.add(keras.Input(shape=(3,)))
model.add(layers.Dense(64, activation="relu"))
```

이제 `build()` 메서드를 실행할 필요 없이 바로 `summary()`를 실행하여
모델의 구조를 확인할 수 있다.

```python
>>> model.summary()
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
dense_4 (Dense)              (None, 64)                256       
=================================================================
Total params: 256
Trainable params: 256
Non-trainable params: 0
_________________________________________________________________
```

층을 추가할 때 마다 모델의 구조를 확인할 수 있다.

```python
>>> model.add(layers.Dense(10, activation="softmax"))
>>> model.summary()
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
dense_4 (Dense)              (None, 64)                256       
_________________________________________________________________
dense_5 (Dense)              (None, 10)                650       
=================================================================
Total params: 906
Trainable params: 906
Non-trainable params: 0
_________________________________________________________________
```

## 케라스 모델 구성법 2: 함수형 API

`Sequential` 클래스를 사용하면 한 종류의 입력값과 한 종류의 출력값만을 사용할 수 있다.
반면에 함수형 API를 이용하면 다중 입력과 다중 출력을 지원하는
모델을 구성할 수 있다.
즉, 모델이 서로 다른 종류의 입력값과 서로 다른 종류의 출력값을 지원한다.

**기본 활용법**

앞서 살펴 본 `Sequential` 모델을 함수형 API를 이용하여 구성하면 다음과 같다.

```python
inputs = keras.Input(shape=(3,), name="my_input")          # 입력층
features = layers.Dense(64, activation="relu")(inputs)     # 은닉층
outputs = layers.Dense(10, activation="softmax")(features) # 출력층
model = keras.Model(inputs=inputs, outputs=outputs)        # 모델 지정
```

사용된 단계들을 하나씩 살펴보자.

- 입력층: 여기서는 한 개만 사용

    ```python
    inputs = keras.Input(shape=(3,), name="my_input")
    ```

- 은닉층: 입력층의 값을 이어 받음. 여러 개의 은닉층을 자유 자재로 구성 가능.

    ```python
    features = layers.Dense(64, activation="relu")(inputs)
    ```

- 출력층: 최종 은닉층의 결과를 이어 받아 출력값 계산. 여기서는 한 개만 사용.

    ```python
    outputs = layers.Dense(10, activation="softmax")(features)
    ```

- 모델 빌드: 입력층과 출력층을 이용하여 모델 지정

    ```python
    model = keras.Model(inputs=inputs, outputs=outputs)
    ```

`summary()` 의 실행결과는 이전과 동일하다.

```python
>>> model.summary()
Model: "functional_1" 
_________________________________________________________________
Layer (type)                 Output Shape              Param # 
=================================================================
my_input (InputLayer)        [(None, 3)]               0 
_________________________________________________________________
dense_6 (Dense)              (None, 64)                256 
_________________________________________________________________
dense_7 (Dense)              (None, 10)                650 
=================================================================
Total params: 906 
Trainable params: 906 
Non-trainable params: 0 
_________________________________________________________________
```

### 다중 입력, 다중 출력 모델

다중 입력과 다중 출력을 지원하는 모델을 구성하는 방법을 예제를 이용하여 설명한다.

:::{prf:example} 고객 요구사항 접수 모델
:label: exp-customer

고객의 요구사항이 입력되었을 때 처리 우선순위와 담당부서를 지정하는 시스템을 구현하려 한다.
고객의 요구사항은 세 종류의 입력값으로 구성된다.
따라서 시스템에 사용될 딥러닝 모델은 세 개의 입력값과 두 개의 출력값을 사용한다. 

입력 사항 세 종류는 다음과 같다.

- `title`: 요구사항 제목
    - 제목을  멀티-핫 인코딩한 0과 1로 구성된 길이가 1만인 벡터. 
    - 1만등 이내의 사용빈도를 갖는 단어만 사용됨.
    - {numref}`%s장 <ch:getting_started_with_neural_networks>`의IMDB 영화 후기 멀티-핫 인코딩 참고
- `text_body`: 요구사항 내용
    - 요구사항 내용을  멀티-핫 인코딩한 0과 1로 구성된 길이가 1만인 벡터. 
    - 1만등 이내의 사용빈도를 갖는 단어만 사용됨.
    - {numref}`%s장 <ch:getting_started_with_neural_networks>`의IMDB 영화 후기 멀티-핫 인코딩 참고
- `tags`: 사용자에 의한 추가 선택 사항
    - 100개 중 여러 개 선택. 
    - 멀티-핫 인코딩된 길이가 1만인 벡터.

출력 사항 두 종류는 다음과 같다.

- `priority`: 요구사항 처리 우선순위. 
    - 0에서 1사이의 값. 
    - `sigmoid` 활성화 함수 활용.
- `department`: 네 개의 요구사항 처리 담당 부서 중 하나 선택. 
    - `softmax` 활성화 함수 활용.
:::

고객 요구사항 접수 모델을 함수형 API를 이용하여 다음과 같이 
구현할 수 있다.

- 입력층: 세 개
    - `title`: 제목 입력 처리
    - `text_body`:  요구사항 내용 입력 처리
    - `tags`: 추가 선택사항 처리

- 은닉층: 두 개
    - `Concatenate` 층: 여러 개의 텐서를 좌우로 이어붙임. 
        여기서는 고객의 요구사항 제목, 내용, 추가 선택사항을 담고 있는
        길이가 각각 1만, 1만, 100인 세 개의 벡터를 이어붙여서 
        길이가 총 2만1백인 하나의 벡터로 변환.
    - `Dense` 층: 64 개의 유닛과 함께 `relu()` 함수 사용
    
- 출력층: 두 개
    - `priority`: 요구사항 처리 우선순위 판단. 0과 1사의 값을 계산하는 `Dense` 층. 
        1개의 유닛과 `sigmoid()` 활성화함수 지정.
    - `department`: 요구사항 처리 담당 부서 선택. 4 개의 부서 각각에 대한 처리 활률값을 계산하는 `Dense` 층. 
        4개의 유닛과 `softmax()` 활성화함수 지정.

```python
vocabulary_size = 10000    # 사용빈도 1만등 인내 단어 사용
num_tags = 100             # 태그 수
num_departments = 4        # 부서 수

# 입력층: 세 개
title = keras.Input(shape=(vocabulary_size,), name="title")
text_body = keras.Input(shape=(vocabulary_size,), name="text_body")
tags = keras.Input(shape=(num_tags,), name="tags")

# 은닉층
features = layers.Concatenate()([title, text_body, tags]) # shape=(None, 10000+10000+100)
features = layers.Dense(64, activation="relu")(features)

# 출력층: 두 개
priority = layers.Dense(1, activation="sigmoid", name="priority")(features)
department = layers.Dense(
    num_departments, activation="softmax", name="department")(features)

# 모델 빌드: 입력값으로 구성된 입력값 리스트와 출력값으로 구성된 출력값 리스트 사용
model = keras.Model(inputs=[title, text_body, tags], outputs=[priority, department])
```

**모델 컴파일**

손실함수와 평가지표 모두 출력층의 개수만큼 지정한다.

- 손실함수(loss)
    - `priority` 대상: `mean_squared_error`
    - `department` 대상: `categorical_crossentropy`

- 평가지표(metrics): 평가지표는 대상 별로 여러 개를 사용할 수 있기에 각각의 대상에 대해 평가지표 리스트를 작성함.
    - `priority` 대상: `["mean_absolute_error", "mean_squared_error"]`
    - `department` 대상: `["accuracy", "AUC", "Precision"]`

```python
model.compile(optimizer="adam",
              loss=["mean_squared_error", "categorical_crossentropy"],
              metrics=[["mean_absolute_error", "mean_squared_error"], ["accuracy", "AUC", "Precision"]])
```

**모델 훈련**

모델 훈련은 `fit()` 함수에 세 개의 훈련 텐서로 이루어진 리스트와 
두 개의 타깃 텐서로 이루어진 리스트를 지정한 후에 실행한다. 

```python
model.fit([title_data, text_body_data, tags_data],
          [priority_data, department_data]
          epochs=10)
```

**모델 평가**

모델 평가도 훈련과 동일한 방식으로 구성된 인자가 사용된다.

```python
model.evaluate([title_data, text_body_data, tags_data],
               [priority_data, department_data])
```

**모델 활용**

예측값은 두 개의 출력층 각각에서 생성된 두 개의 어레이로 구성된 리스트이다.

```python
priority_preds, department_preds = model.predict([title_data, text_body_data, tags_data])
```

- 우선 순위 예측값: 0과 1사이의 확률값

    ```python
    >>> priority_preds
    array([[1.],
           [1.],
           [1.],
           ...,
           [1.],
           [1.],
           [1.]], dtype=float32)
    ```

- 처리 부서 예측값: 각 부서별 적정도를 가리키는 확률값

    ```python
    >>> department_preds
    array([[3.15035926e-04, 9.99676108e-01, 3.48305429e-38, 8.79832805e-06],
           [2.57118052e-04, 9.99649763e-01, 2.81043147e-36, 9.31219838e-05],
           [1.25319435e-04, 9.99730408e-01, 5.14789972e-38, 1.44241771e-04],
           ...,
           [2.07843386e-05, 9.99939203e-01, 6.17548646e-37, 4.00628560e-05],
           [5.92015058e-05, 9.99811471e-01, 3.15372667e-37, 1.29307984e-04],
           [1.07195789e-04, 9.99771416e-01, 1.70482034e-36, 1.21468736e-04]],
          dtype=float32)
    ```

**사전 객체 활용**

입력층과 출력층의 이름을 이용하여 사전 형식으로 입력값과 출력값을 지정할 수 있다.

```python
model.compile(optimizer="adam",
              loss={"priority": "mean_squared_error", "department": "categorical_crossentropy"},
              metrics={"priority": ["mean_absolute_error", "mean_squared_error"], "department": ["accuracy", "AUC", "Precision"]})

model.fit({"title": title_data, "text_body": text_body_data, "tags": tags_data},
          {"priority": priority_data, "department": department_data},
          epochs=1)

model.evaluate({"title": title_data, "text_body": text_body_data, "tags": tags_data},
               {"priority": priority_data, "department": department_data})

priority_preds, department_preds = model.predict(
    {"title": title_data, "text_body": text_body_data, "tags": tags_data})
```

### 모델 구조 그래프

`plot_model()`을 이용하여 층 연결 구조를 그래프로 나타낼 수 있다.

```python
>>> keras.utils.plot_model(model, "ticket_classifier.png")
```

<div align="center"><img src="https://github.com/codingalzi/dlp2/blob/master/jupyter-book/imgs/ch07-ticket_classifier.png?raw=true" style="width:400px;"></div>

:::{admonition} 층 연결 그래프 그리기
:class: info

`keras.utils.plot_model()` 함수가 제대로 작동하려면 
`pydot` 파이썬 모듈과 graphviz 라는 프로그램이 컴퓨터에 설치되어 있어야 한다.

- `pydot` 모듈 설치: `pip install pydot`
- graphviz 프로그램 설치: [https://graphviz.gitlab.io/download/](https://graphviz.gitlab.io/download/)
- 구글 코랩에서는 기본으로 지원됨.
:::

입력 텐서와 출력 텐서의 모양을 함께 표기할 수도 있다.

```python
>>> keras.utils.plot_model(model, "ticket_classifier_with_shape_info.png", show_shapes=True)
```

<div align="center"><img src="https://github.com/codingalzi/dlp2/blob/master/jupyter-book/imgs/ch07-ticket_classifier_with_shapes.png?raw=true" style="width:900px;"></div>

### 모델 재활용

훈련된 모델을 새로운 모델을 구성하는 데에 활용할 수 있다.
먼저 모델의 `layers` 속성을 이용하여 사용된 층에 대한 정보를 확인한다. 
`layers` 속성은 사용된 층들로 이루어진 리스트를 가리킨다.

```python
>>> model.layers
[<keras.src.engine.input_layer.InputLayer at 0x7fc3a1313fd0>,
 <keras.src.engine.input_layer.InputLayer at 0x7fc3a13ce450>,
 <keras.src.engine.input_layer.InputLayer at 0x7fc3a13c5990>,
 <keras.src.layers.merging.concatenate.Concatenate at 0x7fc3a13e0d50>,
 <keras.src.layers.core.dense.Dense at 0x7fc3a13a6310>,
 <keras.src.layers.core.dense.Dense at 0x7fc3a12f6850>,
 <keras.src.layers.core.dense.Dense at 0x7fc3a13e2f90>]
```

예를 들어, 3번 인덱스에 해당하는 층의 입력값과 출력값에 대한 정보는 아래처럼 확인할 수 있다.

```python
>>> model.layers[3].input
[<KerasTensor: shape=(None, 10000) dtype=float32 (created by layer 'title')>,
 <KerasTensor: shape=(None, 10000) dtype=float32 (created by layer 'text_body')>,
 <KerasTensor: shape=(None, 100) dtype=float32 (created by layer 'tags')>]

>>> model.layers[3].output
<KerasTensor: shape=(None, 20100) dtype=float32 (created by layer 'concatenate')>
```

여기서는 문제해결의 어려움 정도를 "quick", "medium", "difficult"로 구분하는 어려움 정도를 판별하는
출력층을 추가하여 총 세 종류의 출력값을 생성하는 모델을 얻고자 한다.
원래는 모델을 새로 구성하고 훈련을 처음부터 다시 시작해야 한다.
하지만 어차피 훈련과정이 이전과 특별히 다를 이유가 없다.
이런 경우 이전에 훈련된 모델의 일부를 재활용할 수 있다.
기존에 훈련된 모델을 재활용하는 경우 
일반적으로 출력층을 제외한 나머지 층을 재활용 대상으로 삼는다.

기존 모델에서 출력층은 5번과 6번 인덱스에 위치하기에 
4번 인덱스가 가리키는 (은닉)층의 출력 정보를 따로 떼어낸다.

```python
>>> features = model.layers[4].output
```

그런 다음 이제 출력층에 문제해결의 어려움 정도를 판별하는 층을 아래와 같이 지정한다.

```python
>>> difficulty = layers.Dense(3, activation="softmax", name="difficulty")(features)
```

새로이 준비된 `'difficulty'` 층을 기존에 사용된 두 개의 출력층과 함께
새로운 모델의 출력층으로 지정한다.

```python
new_model = keras.Model(
    inputs=[title, text_body, tags],
    outputs=[priority, department, difficulty])
```

모델 구성 그래프는 다음과 같다.

```python
>>> keras.utils.plot_model(new_model, "updated_ticket_classifier.png", show_shapes=True)
```

<div align="center"><img src="https://github.com/codingalzi/dlp2/blob/master/jupyter-book/imgs/ch07-ticket_classifier_with_shapes-02.png?raw=true" style="width:900px;"></div>

요약 결과는 다음과 같다.

```python
>>> new_model.summary()
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
==================================================================================================
title (InputLayer)              [(None, 10000)]      0                                            
__________________________________________________________________________________________________
text_body (InputLayer)          [(None, 10000)]      0                                            
__________________________________________________________________________________________________
tags (InputLayer)               [(None, 100)]        0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 20100)        0           title[0][0]                      
                                                                 text_body[0][0]                  
                                                                 tags[0][0]                       
__________________________________________________________________________________________________
dense_8 (Dense)                 (None, 64)           1286464     concatenate[0][0]                
__________________________________________________________________________________________________
priority (Dense)                (None, 1)            65          dense_8[0][0]                    
__________________________________________________________________________________________________
department (Dense)              (None, 4)            260         dense_8[0][0]                    
__________________________________________________________________________________________________
difficulty (Dense)              (None, 3)            195         dense_8[0][0]                    
==================================================================================================
Total params: 1,286,984
Trainable params: 1,286,984
Non-trainable params: 0
__________________________________________________________________________________________________
```

새로 생성된 모델은 출력층을 제외한 나머지 층들이 기존 모델의 훈련 과정에서 학습한
가중치 행렬과 편향 벡터를 그대로 유지한다.

모델 재활용은 기존에 매우 잘 훈련된 모델을 새로운 문제를 해결하는 딥러닝 모델을 
훈련시킬 때 많이 사용하는 기법이다.
{numref}`%s장 <ch:computer-vision-advanced>`에서 모델 재활용을 자세히 다룬다.

## 케라스 모델 구성법 3: 서브클래싱

`keras.Model` 클래스를 상속하는 모델 클래스를 직접 선언하여 활용하는 기법이
**서브클래싱**<font size='2'>subclassing</font>이다.

서브클래싱으로 모델 클래서를 선언하려면 아래 두 메서드를 목적에 맞추어 
재정의<font size='2'>overriding</font>한다.

- `__init__()` 메서드(생성자): 은닉층과 출력층으로 사용될 층 객체 지정
- `call()` 메서드: 층을 연결하는 과정 지정. 즉, 입력값으부터 출력값을 만들어내는 순전파 과정 묘사.

:::{admonition} 모델과 층
:class: note

모델 서브클래싱은 `keras.layers.Layer`를 상속하여 사용자 정의 층 클래스를 선언하는 방식과 
거의 동일하다 ({numref}`%s장 <ch:keras-tf>` 참고).
층 클래스와는 다르게 모델 클래스는 `fit()`, `evaluate()`, `predict()` 메서드를 함께 지원하기에
모델의 훈련, 평가, 예측을 총괄할 수 있다.
물론 모든 과정에서 훈련된 층에 저장된 가중치 행렬과 편향 벡터를 이용한다.
:::

앞서 함수형 API로 구성한 고객 요구사항 접수 모델(두 종류의 출력값 생성 모델)을 서브클래싱을 기법을 이용하여 구현하면 다음과 같다.

```python
class CustomerTicketModel(keras.Model):

    def __init__(self, num_departments): # num_departments: 고객 요구사항 처리 부서 개수
        super().__init__()
        self.concat_layer = layers.Concatenate()                      # 은닉층
        self.mixing_layer = layers.Dense(64, activation="relu")       # 은닉층
        self.priority_scorer = layers.Dense(1, activation="sigmoid")  # 출력층 1
        self.department_classifier = layers.Dense(                    # 출력층 2
            num_departments, activation="softmax")

    def call(self, inputs):               # inputs: 사전 객체 입력값. 모양은 미정.
        title = inputs["title"]
        text_body = inputs["text_body"]
        tags = inputs["tags"]

        features = self.concat_layer([title, text_body, tags])    # 은닉층 연결
        features = self.mixing_layer(features)
        priority = self.priority_scorer(features)                 # 출력층 연결
        department = self.department_classifier(features)
        return priority, department                               # 두 종류의 출력값 지정
```

이전과 동일한 구성의 모델은 다음과 같이 선언한다.

```python
>>> model = CustomerTicketModel(num_departments=4)
```

입력값의 정보가 없기에 층별 가중치와 편향은 초기화되지 않는다.

```python
>>> model.weights
[]
```

컴파일, 훈련, 평가, 예측은 이전과 완전히 동일한 방식으로 실행된다.

**서브클래싱 기법의 장단점**

- 장점
    - `call()` 함수를 이용하여 층을 임의로 구성할 수 있다.
    - `for` 반복문 등 파이썬 프로그래밍 모든 기법을 적용할 수 있다.
- 단점
    - 모델 구성을 전적으로 책임져야 한다.
    - 모델 구성 정보가 `call()` 함수 외부로 노출되지 않아서
        앞서 보았던 그래프 표현을 사용할 수 없다. 

## 케라스 혼합 모델 구성법

소개된 세 가지 방식을 임의로 혼합하여 활용할 수 있다. 

**예제: 서브클래싱 모델을 함수형 모델에 활용하기** (강추!!!)

```python
class Classifier(keras.Model):

    def __init__(self, num_classes=2):
        super().__init__()
        if num_classes == 2:
            num_units = 1
            activation = "sigmoid"
        else:
            num_units = num_classes
            activation = "softmax"
        self.dense = layers.Dense(num_units, activation=activation)

    def call(self, inputs):
        return self.dense(inputs)

inputs = keras.Input(shape=(3,))
features = layers.Dense(64, activation="relu")(inputs)
outputs = Classifier(num_classes=10)(features)
model = keras.Model(inputs=inputs, outputs=outputs)
```

**예제: 함수형 모델을 서브클래싱 모델에 활용하기**

```python
inputs = keras.Input(shape=(64,))
outputs = layers.Dense(1, activation="sigmoid")(inputs)
binary_classifier = keras.Model(inputs=inputs, outputs=outputs)

class MyModel(keras.Model):

    def __init__(self, num_classes=2):
        super().__init__()
        self.dense = layers.Dense(64, activation="relu")
        self.classifier = binary_classifier

    def call(self, inputs):
        features = self.dense(inputs)
        return self.classifier(features)

model = MyModel()
```

## 모델 훈련/평가 방식 조정

케라스 모델의 가장 기본적인 활용법은 아래 과정을 차례대로 이행하는 것이다.

- 컴파일(`compile()`)
- 훈련(`fit()`)
- 평가(`evaluate()`)
- 예측(`predict()`)

하지만 훈련중의 모델 평가 방식과 훈련 과정을 조정할 수 있는 다양한 기법이 지원된다.

- 평가지표<font size='2'>metric</font>를 사용자가 직접 정의할 수 있다.
- 다양한 종류의 콜백<font size='2'>callback</font>을 이용하여
    `fit()` 함수의 훈련 과정을 모니터링 하거나 어느 정도 조정할 수 있다.

### 사용자 정의 평가지표 활용

훈련중인 모델의 성능을 평가하는 평가지표<font size='2'>metric</font>는 문제에 따라 일반적으로 사용되는 것들이 있다.

- 회귀 모델: 평균제곱근오차(RMSE), 평균절대오차(MAE) 등
- 분류 모델: 정확도, 정밀도 등

하지만 필요에 따라 사용자가 직접 모델 평가지표를 정의해서 활용할 수 있다.
케라스와 호환되는 평가지표를 정의하기 위해서는 `keras.metrics.Metric` 클래스를 상속하면 된다.

### 콜백 활용

**콜백**<font size='2'>callback</font>은 모델 훈련 과정중에
저장되는 모든 기록을 모니터링하면서 필요에 따라 특정 기능을 수행한다.
가장 많이 활용되는 콜백의 기능과 담당 콜백 클래스는 다음과 같다.

- 훈련중인 모델의 상태 저장: 예를 들어, 훈련 중 가장 좋은 성능의 모델(의 상태) 저장한다.
    모델의 **상태**<font size='2'>state</font>는 훈련 중인 모델에 저장된 가중치와 편향 등의 파라미터를 가리킨다.
    - `keras.callbacks.ModelCheckpoint`

- 훈련 조기 종료: 검증셋에 대한 손실이 더 이상 개선되지 않는 경우 훈련 종료시킨다.
    - `keras.callbacks.EarlyStopping`

- 하이퍼 파라미터 조정: 예를 들어 학습률을 훈련 과정중에 동적으로 변경한다.
    - `keras.callbacks.LearningRateScheduler`
    - `keras.callbacks.ReduceLROnPlateau`

- 훈련 기록 작성: 훈련셋, 검증셋에 대한 손실값, 평가지표 등을 기록하고 시각화한다.
    예를 들어, `fit()` 함수가 호출되어 훈련이 진행중일 때 에포크마다 보여지는 손실값, 평가지표 등을 관리한다.
    - `keras.callbacks.CSVLogger`

:::{prf:example} `EarlyStopping`과 `ModelCheckpoint` 활용
:label: exp-callbacks

아래 코드는 `fit()` 함수 호출에 다음 두 종류의 콜백을 사용하는 방식을 보여준다.

- `EarlyStopping`: 검증셋에 대한 정확도가 2 에포크 연속 개선되지 않을 때 훈련을 종료시킨다.
- `ModelCheckpoint`: 매 에포크마다 훈련된 모델을 저장한다.
    `save_best_only=True`가 설정된 경우 검증셋에 대한 손실값이 가장 낮은 모델, 
    즉 그때까지 훈련 모델 중에서 가장 성능이 좋은 모델만 저장한다.

```python
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=2,
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="checkpoint_path",
        monitor="val_loss",
        save_best_only=True,
    )
]

model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
model.fit(train_images, train_labels,
          epochs=10,
          callbacks=callbacks_list,
          validation_data=(val_images, val_labels))
```
:::

### 사용자 정의 콜백 활용

케라스와 호환되는 콜백 클래스를 정의하려면 `keras.callbacks.Callback` 클래스를 상속하면 된다.

### 텐서보드(TensorBoard) 활용

**텐서보드**<font size='2'>TensorBoard</font>는 모델 훈련과정을 모니터링하는 최고의 어플이며
텐서플로우와 함께 기본적으로 설치된다.

<div align="center"><img src="https://github.com/codingalzi/dlp2/blob/master/jupyter-book/imgs/ch07-tensorboard.png?raw=true" style="width:600px;"></div>

## 훈련/평가 알고리즘 직접 구현하기

모델 컴파일 이후 `fit()` 메서드를 호출하면 모델의 훈련이 진행된다.
그런데 모델의 훈련 방식을 사용자가 직접 조정할 수 있는 방식으로 진행하고자 하면
아래 과정을 자신만의 알고리즘으로 직접 구현하면 된다.

- 순전파<font size='2'>forward pass</font>
- 손실함수의 그레이디언트 계산
- 역전파<font size='2'>backward pass</font>

자세한 이야기는 [(구글 코랩) 케라스 모델 활용법](https://colab.research.google.com/github/codingalzi/dlp2/blob/master/notebooks/NB-working_with_keras.ipynb)을 참고한다.

## 연습문제

1. [(실습) 케라스 모델 고급 활용법](https://colab.research.google.com/github/codingalzi/dlp2/blob/master/excs/exc-working_with_keras.ipynb)